In [ ]:
!unzip /content/66e31d6ee96cd_student_resource_3.zip

Archive:  /content/66e31d6ee96cd_student_resource_3.zip
   creating: student_resource 3/
  inflating: __MACOSX/._student_resource 3  
  inflating: student_resource 3/sample_code.py  
  inflating: __MACOSX/student_resource 3/._sample_code.py  
  inflating: student_resource 3/.DS_Store  
  inflating: __MACOSX/student_resource 3/._.DS_Store  
   creating: student_resource 3/dataset/
  inflating: __MACOSX/student_resource 3/._dataset  
  inflating: student_resource 3/README.md  
  inflating: __MACOSX/student_resource 3/._README.md  
   creating: student_resource 3/src/
  inflating: __MACOSX/student_resource 3/._src  
  inflating: student_resource 3/dataset/sample_test_out.csv  
  inflating: __MACOSX/student_resource 3/dataset/._sample_test_out.csv  
  inflating: student_resource 3/dataset/sample_test_out_fail.csv  
  inflating: __MACOSX/student_resource 3/dataset/._sample_test_out_fail.csv  
  inflating: student_resource 3/dataset/test.csv  
  inflating: __MACOSX/student_resource 3/dataset

In [ ]:
import torch
from torch import nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn

In [ ]:
df=pd.read_csv("/content/student_resource 3/dataset/train.csv")

In [ ]:
df.head()

,image_link,group_id,entity_name,entity_value
0,https://m.media-amazon.com/images/I/61I9XdN6OF...,748919,item_weight,500.0 gram
1,https://m.media-amazon.com/images/I/71gSRbyXmo...,916768,item_volume,1.0 cup
2,https://m.media-amazon.com/images/I/61BZ4zrjZX...,459516,item_weight,0.709 gram
3,https://m.media-amazon.com/images/I/612mrlqiI4...,459516,item_weight,0.709 gram
4,https://m.media-amazon.com/images/I/617Tl40LOX...,731432,item_weight,1400 milligram


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 263859 entries, 0 to 263858
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   image_link    263859 non-null  object
 1   group_id      263859 non-null  int64 
 2   entity_name   263859 non-null  object
 3   entity_value  263859 non-null  object
dtypes: int64(1), object(3)
memory usage: 8.1+ MB


In [ ]:
df['entity_name'].unique()

array(['item_weight', 'item_volume', 'voltage', 'wattage',
       'maximum_weight_recommendation', 'height', 'depth', 'width'],
      dtype=object)

In [ ]:
maxlen_entity_value=max([len(tex) for tex in df['entity_value'].unique()])

In [ ]:
df.isnull().sum()

,0
image_link,0
group_id,0
entity_name,0
entity_value,0


In [ ]:
df['image_link'][0]

'https://m.media-amazon.com/images/I/61I9XdN6OFL.jpg'

In [ ]:
len(df['image_link'])

263859

In [ ]:
test_df=pd.read_csv("/content/student_resource 3/dataset/test.csv")

In [ ]:
test_df.head()

,index,image_link,group_id,entity_name
0,0,https://m.media-amazon.com/images/I/110EibNycl...,156839,height
1,1,https://m.media-amazon.com/images/I/11TU2clswz...,792578,width
2,2,https://m.media-amazon.com/images/I/11TU2clswz...,792578,height
3,3,https://m.media-amazon.com/images/I/11TU2clswz...,792578,depth
4,4,https://m.media-amazon.com/images/I/11gHj8dhhr...,792578,depth


In [ ]:
test_df['entity_name'].unique()

array(['height', 'width', 'depth', 'item_weight',
       'maximum_weight_recommendation', 'wattage', 'voltage',
       'item_volume'], dtype=object)

In [ ]:
test_df['entity_value']=''

In [ ]:
entity_unit_map = {
    'width': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'depth': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'height': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'item_weight': {'gram',
        'kilogram',
        'microgram',
        'milligram',
        'ounce',
        'pound',
        'ton'},
    'maximum_weight_recommendation': {'gram',
        'kilogram',
        'microgram',
        'milligram',
        'ounce',
        'pound',
        'ton'},
    'voltage': {'kilovolt', 'millivolt', 'volt'},
    'wattage': {'kilowatt', 'watt'},
    'item_volume': {'centilitre',
        'cubic foot',
        'cubic inch',
        'cup',
        'decilitre',
        'fluid ounce',
        'gallon',
        'imperial gallon',
        'litre',
        'microlitre',
        'millilitre',
        'pint',
        'quart'}
}

allowed_units = {unit for entity in entity_unit_map for unit in entity_unit_map[entity]}

In [ ]:
import re
import os
import requests
import pandas as pd
import multiprocessing
import time
from time import time as timer
from tqdm import tqdm
import numpy as np
from pathlib import Path
from functools import partial
import requests
import urllib
from PIL import Image

def common_mistake(unit):
    if unit in allowed_units:
        return unit
    if unit.replace('ter', 'tre') in allowed_units:
        return unit.replace('ter', 'tre')
    if unit.replace('feet', 'foot') in allowed_units:
        return unit.replace('feet', 'foot')
    return unit

def parse_string(s):
    s_stripped = "" if s==None or str(s)=='nan' else s.strip()
    if s_stripped == "":
        return None, None
    pattern = re.compile(r'^-?\d+(\.\d+)?\s+[a-zA-Z\s]+$')
    if not pattern.match(s_stripped):
        raise ValueError("Invalid format in {}".format(s))
    parts = s_stripped.split(maxsplit=1)
    number = float(parts[0])
    unit = common_mistake(parts[1])
    if unit not in allowed_units:
        raise ValueError("Invalid unit [{}] found in {}. Allowed units: {}".format(
            unit, s, allowed_units))
    return number, unit


def create_placeholder_image(image_save_path):
    try:
        placeholder_image = Image.new('RGB', (100, 100), color='black')
        placeholder_image.save(image_save_path)
    except Exception as e:
        return

def download_image(image_link, save_folder, retries=3, delay=3):
    if not isinstance(image_link, str):
        return

    filename = Path(image_link).name
    image_save_path = os.path.join(save_folder, filename)

    if os.path.exists(image_save_path):
        return

    for _ in range(retries):
        try:
            urllib.request.urlretrieve(image_link, image_save_path)
            return
        except:
            time.sleep(delay)

    create_placeholder_image(image_save_path) #Create a black placeholder image for invalid links/images

def download_images(image_links, download_folder, allow_multiprocessing=True):
    if not os.path.exists(download_folder):
        os.makedirs(download_folder)

    if allow_multiprocessing:
        download_image_partial = partial(
            download_image, save_folder=download_folder, retries=3, delay=3)

        with multiprocessing.Pool(64) as pool:
            list(tqdm(pool.imap(download_image_partial, image_links), total=len(image_links)))
            pool.close()
            pool.join()
    else:
        for image_link in tqdm(image_links, total=len(image_links)):
            download_image(image_link, save_folder=download_folder, retries=3, delay=3)


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
download_images(test_df['image_link'],'/content/student_resource 3/dataset/test_img_1')

100%|██████████| 131187/131187 [14:18<00:00, 152.76it/s]


In [ ]:
unq_l=""
for i in df["entity_value"].unique():
    #print(i)
    for j in i:
        if j not in unq_l:
            unq_l+=j
print(sorted(unq_l))

from sklearn.preprocessing import LabelEncoder
target_chars = sorted(unq_l+'xz')
target_chars=target_chars+['<sos>']+['<eos>']+['<pad>']
target_encoder = LabelEncoder()
target_encoder.fit(target_chars)

question_encoder = LabelEncoder()
question_encoder.fit(df['entity_name'].values)

[' ', '+', ',', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'y']


LabelEncoder()

In [ ]:
from torchvision import models

In [ ]:
class Encoder(torch.nn.Module):
    def __init__(self,img_dim,total_questions):
        super(Encoder,self).__init__()
        self.img_dim = img_dim
        self.image_model = models.resnet50(pretrained=True)
        self.img_projection = torch.nn.Linear(2048,self.img_dim)
        self.ques_embedding = torch.nn.Embedding(total_questions,128)
        self.image_model.fc=torch.nn.Identity()
        for param in self.image_model.parameters():
            param.requires_grad = False

    def forward(self,inputs):
        image_array = inputs['image']
        img = self.image_model(image_array)#batch,128
        img = torch.nn.functional.relu(self.img_projection(img))
        question = inputs['question']
        q = self.ques_embedding(question)#batch,1,128
        q = torch.squeeze(q,1)#batch,128
        return torch.cat((img,q),1)#batch,128+128


class Decoder(torch.nn.Module):
    def __init__(self,emd_dim,hidden_size,output_chars):
        super(Decoder,self).__init__()
        self.emd_dim = emd_dim
        self.embedding = torch.nn.Embedding(output_chars,self.emd_dim)
        self.lstm = torch.nn.LSTM(self.emd_dim,hidden_size,num_layers=1,batch_first=True)
        self.fc = torch.nn.Linear(hidden_size,output_chars)

    def forward(self,inputs,hidden_state,cell):
        x = self.embedding(inputs)
        #print(f'x: {x.shape}')
        x,(hidden,cell) = self.lstm(x,(hidden_state,cell))
        #print(f'from decoder x: {x.shape}')
        output = self.fc(x)
        return output,hidden,cell

class Final_Model(torch.nn.Module):
    def __init__(self,img_dim,emd_dim,target_length,total_chars,batch_size,device):
        super(Final_Model,self).__init__()
        self.target_length = target_length
        self.device = device
        self.batch_size = batch_size
        self.total_chars = total_chars
        self.encoder = Encoder(img_dim,8)
        self.decoder = Decoder(emd_dim,256,total_chars)

    def forward(self,inputs):
        target = inputs['target']
        hidden = cell = self.encoder(inputs) #Batch,256
        hidden = torch.unsqueeze(hidden,0)
        cell = torch.unsqueeze(cell,0)
        final_tokens = torch.zeros(self.target_length,self.batch_size,self.total_chars).to(self.device)
        for i in range(1,self.target_length):
            #target: batch, target_length
            # first token: sos, real taregt, eos,pad.....
            prev_pred = torch.unsqueeze(target[...,i-1],-1)
            curr_pred,hidden,cell = self.decoder(prev_pred,hidden,cell)
            #curr_pred: batch, 1, total_chars
            final_tokens[i] = torch.squeeze(curr_pred,1)
        return final_tokens

final_model = Final_Model(128,128,35,44,64,device)
final_model.load_state_dict(torch.load('/content/saving_model_0.pt',map_location=device))

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 168MB/s]
<ipython-input-23-bfaa9897ddcf>:64: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is poss

<All keys matched successfully>

In [ ]:
from torch.utils.data import DataLoader
from torchvision import transforms
import PIL

In [ ]:
class CustomDataset(torch.utils.data.dataset.Dataset):
    def __init__(self,dataset,transform,source_dir,target_length,entity_transform,target_encoder):
        self.dataset = dataset
        self.transform = transform
        self.source_dir = source_dir
        self.entity_transform = entity_transform
        self.target_encoder= target_encoder
        self.target_length = target_length

    def __len__(self):
        return self.dataset.shape[0]

    def __get_image(self,path):
        img_array = PIL.Image.open(path)
        transformed_img = self.transform(img_array)
        #print(f'transimage: {transformed_img.shape}')
        if transformed_img.shape[0]==1:
            transformed_img = transformed_img.repeat(3,1,1)
        return transformed_img

    def __create_target(self,target):
        char_lvl = ['<sos>']+list(s for s in target)+['<eos>']
        extra_length = self.target_length - len(char_lvl)
        if extra_length>=0:
            char_lvl = char_lvl + ['<pad>']*extra_length
        else:
            char_lvl = char_lvl[:self.target_length -1]+['<eos>']
        #print(f'char lvl: {char_lvl}')
        return self.target_encoder.transform(char_lvl)

    def __getitem__(self,idx):
        image_name = self.dataset.loc[idx,'image_link'].split('/')[-1]
        img_path = self.source_dir+'/'+image_name#os.path.join(self.source_dir,image_name)
        img_array = self.__get_image(img_path)
        #print("image got!")
        entity_name = self.dataset.loc[idx,'entity_name']
        entity_vector = np.array(self.entity_transform.transform([entity_name]),dtype=np.int32)
        entity_value = self.dataset.loc[idx,'entity_value']
        target = self.__create_target(entity_value)
        return {"image":img_array,"question":entity_vector,"target":target,'path':img_path,'entity_name':entity_name}

test_df['entity_value'] = ''
data_transform=transforms.Compose([
    transforms.Resize((256,256)),
    transforms.ToTensor()
])

test_dir= '/content/student_resource 3/dataset/test_img_1'
val_dataset = CustomDataset(test_df,data_transform,test_dir,35,question_encoder,target_encoder)
val_dataloader = DataLoader(val_dataset,batch_size=64,shuffle=False,drop_last=False)
final_results = {"img_path":[],"prediction":[],'entity_name':[]}
test_iter = iter(val_dataloader)

In [ ]:
final_model = torch.compile(final_model)

In [ ]:
for batch_idx in tqdm(range(len(val_dataloader))):
    curr_batch = next(test_iter)
    preds = final_model(curr_batch)
    img_paths = curr_batch['path']
    questions = curr_batch['entity_name']
    batch_preds = torch.argmax(torch.nn.functional.softmax(preds),-1).permute(1,0).cpu().numpy()
    for i in range(batch_preds.shape[0]):
        prediction = batch_preds[i,...]
        list_form = target_encoder.inverse_transform(prediction).tolist()
        try:
            eos_index = list_form.index('<eos>')
        except:
            eos_index = len(list_form)
        list_form = list_form[:eos_index]
        string_pred = ''.join(list_form)
        string_pred = string_pred.replace('<sos>',"")
        string_pred = string_pred.replace('<pad>',"")
        final_results['img_path'].append(img_paths[i])
        final_results['entity_name'].append(questions[i])
        final_results['prediction'].append(string_pred)

  0%|          | 0/2050 [00:00<?, ?it/s]<ipython-input-39-f96c9fcbeb34>:6: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  batch_preds = torch.argmax(torch.nn.functional.softmax(preds),-1).permute(1,0).cpu().numpy()
  0%|          | 1/2050 [01:38<56:20:20, 98.99s/it]<ipython-input-39-f96c9fcbeb34>:6: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  batch_preds = torch.argmax(torch.nn.functional.softmax(preds),-1).permute(1,0).cpu().numpy()
  0%|          | 2/2050 [01:50<27:08:27, 47.71s/it]<ipython-input-39-f96c9fcbeb34>:6: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  batch_preds = torch.argmax(torch.nn.functional.softmax(preds),-1).permute(1,0).cpu().numpy()
  0%|          | 3/2050 [02:01<17:36:11, 30.96s/it]<ipython-input-39-f96c9fcbeb34>:6: UserWarning: Implic

In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [ ]:
data_transform=transforms.Compose([
    transforms.Resize((256,256)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])


In [ ]:
from torchvision import datasets
train_data=datasets.ImageFolder(root=train_dir,
                                transform=data_transform,
                                target_transform=None)
test_data=datasets.ImageFolder(root=test_dir,
                               transform=data_transform)

In [ ]:
train_data

In [ ]:
import os
from torch.utils.data import DataLoader
train_dataloader=DataLoader(dataset=train_data,
                            batch_size=32,
                            num_workers=os.cpu_count(),
                            shuffle=True)
test_dataloader=DataLoader(dataset=test_data,
                           batch_size=32,
                           num_workers=os.cpu_count(),
                           shuffle=False)
train_dataloader,test_dataloader

In [ ]:
from torchvision import models
model=models.resnet50(pretrained=True)

In [ ]:
model

In [ ]:
model.fc = nn.Identity()

In [ ]:
model

In [ ]:
'''
def test_step(model:torch.nn.Module,
              dataloader:torch.utils.data.DataLoader):
  model.eval()
  model.to(device)
  with torch.inference_mode():
    for batch,(X,y) in enumerate(dataloader):
      X,y=X.to(device),y.to(device)
      test_pred=model(X)
  return test_pred
'''

**error**:The code is trying to load an image file that is corrupted or incomplete, hence the "image file is truncated" error. This is happening within the DataLoader worker process, which suggests that the issue lies with one of the images in your training dataset.

!pip install Pillow
from PIL import ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES = True
def test_step(model:torch.nn.Module,
              dataloader:torch.utils.data.DataLoader):
  model.eval()
  model.to(device)
  with torch.inference_mode():
    for batch,(X,y) in enumerate(dataloader):
      X,y=X.to(device),y.to(device)
      test_pred=model(X)
  return test_pred

This code snippet tells PIL (Pillow) to ignore the truncated data and load the image anyway. This might allow your code to continue running, but be aware that the truncated images might not be loaded correctly and could lead to unexpected results in your model training or testing.

If setting LOAD_TRUNCATED_IMAGES to True doesn't solve the problem or leads to other issues, you may need to identify and fix or remove the corrupted image(s) from your dataset.


In [ ]:
!pip install Pillow
from PIL import ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES = True
def test_step(model:torch.nn.Module,
              dataloader:torch.utils.data.DataLoader):
  model.eval()
  model.to(device)
  with torch.inference_mode():
    for batch,(X,y) in enumerate(dataloader):
      X,y=X.to(device),y.to(device)
      test_pred=model(X)
  return test_pred

In [ ]:
test_step(model=model,dataloader=train_dataloader)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
ohe=OneHotEncoder()

In [ ]:
ohe.fit_transform(sample_data['entity_name']).toarray()

In [ ]:
sample_data={
    'entity_name':['height', 'width', 'depth', 'item_weight',
       'maximum_weight_recommendation', 'wattage', 'voltage',
       'item_volume']
}

In [ ]:
sample_data[:,0]=encode.fit_transform(sample_data[:,0])